# Siamese network implementation with the strips dataset

In [1]:
import os
import numpy as np
import random as rng
import cv2
import json

In [2]:
info_data = []
with open('/home/data/strips_socrates/dataset_info.json') as json_file: 
    info_data = json.load(json_file) 

In [3]:
STRIP_SIZE = 256
SUB_STRIP_SIZE = 128

In [4]:
def get_batch(info_data, batch_size, with_id=False, dataset = 'train'):
    pairs = [np.zeros((batch_size, STRIP_SIZE, STRIP_SIZE, 3)) for i in range(2)]
    labels = np.zeros((batch_size, ))
    labels[batch_size//2:] = 1
    split_index = int((len(info_data)-1) * 0.8)
    if dataset == 'train':
        students = [rng.randint(0, split_index) for _ in range(batch_size)] 
    elif dataset == 'whole':
        students = [rng.randint(0, (len(info_data)-1)) for _ in range(batch_size)] 
    else:
        students = [rng.randint(split_index+1, len(info_data)-1) for _ in range(batch_size)] 
    imgs = [rng.randint(0, len(info_data[i])-1) for i in students]
    strips_loc = [rng.randint(0, len(info_data[i][0][1])-1) for i in students]
    id_pairs = []
    for i in range(batch_size):
        std = students[i]
        img = imgs[i]
        srl = strips_loc[i]
        strip1 = info_data[std][img][1][srl]
        strip2 = ""
        if i >= batch_size // 2:
            img2 = (img + rng.randint(1, len(info_data[std])-1)) % len(info_data[std])
            strip2 = info_data[std][img2][1][srl]
        else:
            std2 = (std + rng.randint(1, len(info_data)-1)) % len(info_data)
            img2 = rng.randint(0, len(info_data[std2])-1)
            srl2 = rng.randint(0, len(info_data[std2][0][1])-1)
            strip2 = info_data[std2][img2][1][srl2]
        pairs[0][i,:,:,:] = cv2.imread(strip1)/255
        pairs[1][i,:,:,:] = cv2.imread(strip2)/255
        id_pairs.append((strip1, strip2))
    if with_id:
        return pairs, labels, id_pairs
    else:
        return pairs, labels

In [ ]:
p,l,ids = get_batch(info_data, 10, True)

In [ ]:
for x in zip(l, ids):
    print(x)

In [5]:
def generate(info_data, batch_size, dataset='train'):
    while True:
        pairs, labels = get_batch(info_data, batch_size, False, dataset=dataset)
        yield(pairs, labels)

In [6]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import backend as K

In [7]:
def sequential_block(input_shape = (256,256,3), base_filters=64):
    inputs = layers.Input(shape=input_shape)
    model = keras.applications.EfficientNetB1(include_top=False, input_tensor=inputs, weights=None)

    

    # Rebuild top
    x = layers.GlobalAveragePooling2D(name="avg_pool")(model.output)
    x = layers.BatchNormalization()(x)
    
    
    top_dropout_rate = 0.2
    x = layers.Dropout(top_dropout_rate, name="top_dropout")(x)
    outputs = layers.Dense(1024, activation="softmax", name="pred")(x)
    return keras.Model(inputs, outputs)

In [ ]:
seq = sequential_block(base_filters=8)

In [ ]:
seq.summary()

In [ ]:
seq = None

In [8]:
def get_siamese_model(input_shape = (256,256,3), base_filters=16):

    left_input = layers.Input(input_shape)
    right_input = layers.Input(input_shape)
    
    # Convolutional Neural Network
    model = sequential_block(input_shape, base_filters)
    
    # Generate the encodings (feature vectors) for the two images
    encoded_l = model(left_input)
    encoded_r = model(right_input)
    
    # Add a customized layer to compute the absolute difference between the encodings
    L1_layer = layers.Lambda(lambda tensors:K.abs(tensors[0] - tensors[1]))
    L1_distance = L1_layer([encoded_l, encoded_r])
    
    # Add a dense layer with a sigmoid unit to generate the similarity score
    prediction = layers.Dense(1,activation='sigmoid')(L1_distance)
    
    # Connect the inputs with the outputs
    siamese_net = keras.Model(inputs=[left_input,right_input],outputs=prediction)
    
    # return the model
    return siamese_net

In [ ]:
def get_siamese_model_ensemble(input_shape = (STRIP_SIZE,STRIP_SIZE,3), base_filters=16):

    left_input = layers.Input(input_shape)
    left_A = layers.Cropping2D(cropping=((0,SUB_STRIP_SIZE), (0,SUB_STRIP_SIZE)))(left_input)
    left_B = layers.Cropping2D(cropping=((SUB_STRIP_SIZE,0), (0,SUB_STRIP_SIZE)))(left_input)
    left_C = layers.Cropping2D(cropping=((0,SUB_STRIP_SIZE), (SUB_STRIP_SIZE,0)))(left_input)
    left_D = layers.Cropping2D(cropping=((SUB_STRIP_SIZE,0), (SUB_STRIP_SIZE,0)))(left_input)
    right_input = layers.Input(input_shape)
    right_A = layers.Cropping2D(cropping=((0,SUB_STRIP_SIZE), (0,SUB_STRIP_SIZE)))(right_input)
    right_B = layers.Cropping2D(cropping=((SUB_STRIP_SIZE,0), (0,SUB_STRIP_SIZE)))(right_input)
    right_C = layers.Cropping2D(cropping=((0,SUB_STRIP_SIZE), (SUB_STRIP_SIZE,0)))(right_input)
    right_D = layers.Cropping2D(cropping=((SUB_STRIP_SIZE,0), (SUB_STRIP_SIZE,0)))(right_input)
    
    
    siam = get_siamese_model((SUB_STRIP_SIZE,SUB_STRIP_SIZE,3), base_filters=base_filters)
    out_A = siam([left_A, right_A]) 
    out_B = siam([left_B, right_B]) 
    out_C = siam([left_C, right_C]) 
    out_D = siam([left_D, right_D]) 
    
    prediction = layers.Average()([out_A, out_B, out_C, out_D])
    
    # Connect the inputs with the outputs
    siamese_net = keras.Model(inputs=[left_input,right_input],outputs=prediction)
    
    # return the model
    return siamese_net

In [9]:
model = get_siamese_model(base_filters=64)
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 256, 256, 3) 0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 256, 256, 3) 0                                            
__________________________________________________________________________________________________
model (Functional)              (None, 1024)         7892103     input_1[0][0]                    
                                                                 input_2[0][0]                    
__________________________________________________________________________________________________
lambda (Lambda)                 (None, 1024)         0           model[0][0]                

In [31]:
initial_learning_rate = 1e-6
end_learning_rate = 1e-7
decay_steps = 12800
lr_schedule = keras.optimizers.schedules.PolynomialDecay(
    initial_learning_rate, decay_steps, end_learning_rate, power=0.5
)
model.compile(
        loss="binary_crossentropy",
        optimizer=keras.optimizers.Adam(learning_rate=lr_schedule),
        metrics=[keras.metrics.Precision(name='precision'),
             keras.metrics.Recall(name='recall'), "accuracy"] 
    )
run_name = "enet-test03-withval-propper"
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir="./logs/"+run_name)

In [32]:
batch_size = 8
epochs = 280
steps_epoch = 128

In [33]:
model.fit(generate(info_data, batch_size), 
          epochs = epochs, 
          steps_per_epoch= steps_epoch,
          callbacks=[tensorboard_callback],
          validation_data=val_set,
          validation_batch_size=8,
          shuffle = True)

Epoch 1/280
128/128 [==============================] - 109s 707ms/step - loss: 0.6870 - precision: 0.5643 - recall: 0.5540 - accuracy: 0.5627 - val_loss: 0.6862 - val_precision: 0.5701 - val_recall: 0.6100 - val_accuracy: 0.5750
Epoch 2/280
128/128 [==============================] - 84s 658ms/step - loss: 0.6806 - precision: 0.5989 - recall: 0.6494 - accuracy: 0.6075 - val_loss: 0.6906 - val_precision: 0.5490 - val_recall: 0.5600 - val_accuracy: 0.5500
Epoch 3/280
128/128 [==============================] - 85s 660ms/step - loss: 0.6855 - precision: 0.5788 - recall: 0.5805 - accuracy: 0.5786 - val_loss: 0.6870 - val_precision: 0.5660 - val_recall: 0.6000 - val_accuracy: 0.5700
Epoch 4/280
128/128 [==============================] - 84s 660ms/step - loss: 0.6914 - precision: 0.5218 - recall: 0.5747 - accuracy: 0.5225 - val_loss: 0.6901 - val_precision: 0.5455 - val_recall: 0.5400 - val_accuracy: 0.5450
Epoch 5/280
128/128 [==============================] - 84s 659ms/step - loss: 0.6878 - 

In [18]:
model.save("enet-02.h5")

In [ ]:
model.load_weights("test06-interrupt.h5")

### Validation:
Simple, just feed pairs from the validation split and see the performance.

In [29]:
val_set = get_batch(info_data, 200, False, 'validation')

In [ ]:
preds = model.predict(val_set_X)

In [ ]:
corr = [y == (p>0) for y, p in zip (val_set_y, preds)]

In [ ]:
round(preds[0][0])

In [ ]:
cs = np.zeros((2,2))
for y,p in zip (val_set_y, preds):
    cs[int(y), round(p[0])] += 1 
acc = (cs[0,0] + cs [1 , 1]) / np.sum(cs)
rec = cs[1,1] / np.sum(cs[1])
print(cs, acc, rec)

### Extended validation:
Take two images, and compare strip by strip, then do a majority voting.

In [ ]:
def get_val_batch(info_data, num_whole_images, batch_size):
    images = []
    split_index = int((len(info_data)-1) * 0.8)
    students = [rng.randint(split_index+1, len(info_data)-1) for _ in range(num_whole_images)] 
    labels = np.zeros((num_whole_images, ))
    labels[num_whole_images//2:] = 1
    id_pairs_img = []
    for j in range(num_whole_images):
        pairs = [np.zeros((batch_size, STRIP_SIZE, STRIP_SIZE, 3)) for i in range(2)]
        imgs = [rng.randint(0, len(info_data[i])-1) for i in students]
        std = students[j]
        img = imgs[j]
        id_pairs = []
        std2 = (std + rng.randint(1, len(info_data)-1)) % len(info_data)
        if j >= num_whole_images // 2:   
            img2 = (img + rng.randint(1, len(info_data[std])-1)) % len(info_data[std])
            std2 = std
        else:
            std2 = (std + rng.randint(1, len(info_data)-1)) % len(info_data)
            img2 = rng.randint(0, len(info_data[std2])-1)
        if batch_size > len(info_data[std][img][1]):
            print("a", info_data[std][0][0])
            std = std+1
            
        if batch_size > len(info_data[std2][img2][1]):
            print("b", info_data[std2][0][0])
            std2 = std+1
            
        for i in range(batch_size):    
            strip1 = info_data[std][img][1][i]
            strip2 = info_data[std2][img2][1][i]
#             print(strip1, strip2)
            pairs[0][i,:,:,:] = cv2.imread(strip1)/255
            pairs[1][i,:,:,:] = cv2.imread(strip2)/255
            id_pairs.append((strip1, strip2))
        images.append(pairs)
        id_pairs_img.append(id_pairs)
    
    return images, labels, id_pairs_img

In [ ]:
t_i, t_l, t_d = get_val_batch(info_data, 100, 10)

In [ ]:
cs = np.zeros((2,2))
cs_all = np.zeros((2,2))
for i, l, d in zip(t_i, t_l, t_d):
    preds = model.predict(i)
    for p in preds:
        cs_all[int(l), round(p[0])] += 1
#     print(int(l), round(np.mean(preds)), (np.mean(preds)))
    fp = sum([p[0] > 0.5 for p in preds]) > len(preds)//2 -2
    cs[int(l), int(fp)] +=1
#     print(l, d[0])

acc = (cs[0,0] + cs [1 , 1]) / np.sum(cs)
rec = cs[1,1] / np.sum(cs[1])
print(cs, acc, rec)
cs = cs_all
acc = (cs[0,0] + cs [1 , 1]) / np.sum(cs)
rec = cs[1,1] / np.sum(cs[1])
print(cs, acc, rec)